Programming Fundamentals
============================

In this notebook, emphasis will be placed on data preprocessing. Inconsistencies and missing values will be addressed using numpy and pandas libraries, and sklearn library will be used to transform some data. 

## Exercice 1

Alcohol consumption is a characteristic feature of each country often related to certain social habits. In the `drinks.csv` file there is the alcohol consumption per country considering three tipes of drinks: beer, wine and spirit drinks. In addition, the total amount of pure alcohol consumed in the country in liters can also be found. The original dataset can be dowloaded in [FiveThirtyEight](https://github.com/fivethirtyeight/data/tree/master/alcohol-consumption), however, we will use a modified version.

**a)** Create a dataframe using the dataset from the `data` folder. Show the total number of measures, the name of the variables and the 10 first rows. 


In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("data/drinks.csv", sep=",")

print(data.shape)
print(data.columns)
data.head(n=10)

(169, 7)
Index(['country', 'iso3', 'continent', 'beer_servings', 'spirit_servings',
       'wine_servings', 'total_litres_of_pure_alcohol'],
      dtype='object')


,country,iso3,continent,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,AFG,Asia,0,0,0,0.0
1,Albania,ALB,Europa,89,132,54,4.9
2,Algeria,DZA,Africa,25,0,14,0.7
3,Andorra,AND,Europa,245,138,312,12.4
4,Angola,AGO,Africa,217,57,45,5.9
5,Antigua & Barbuda,ATG,América,102,128,45,4.9
6,Argentina,ARG,América,193,25,221,8.3
7,Armenia,ARM,Asia,21,179,11,3.8
8,Australia,AUS,Oceanía,261,72,212,10.4
9,Austria,AUT,Europa,279,75,191,9.7


**b)** How many possible values has the variable `continent`? Unify the possible values to: "Asia", "Africa", "Oceania", "Europa" i "America".

In [2]:
print("Number of possible values: {}\n".format(len(data["continent"].unique())))
print(data["continent"].unique())

Number of possible values: 18

['Asia' 'Europa' 'Africa' 'América' 'Oceanía' 'America' 'africa' 'ASIA'
 'África' 'AMERICA' 'OceANia' 'Oceania' 'Europa ' 'AMERICa ' 'Oceaniaa'
 'AFRICA' 'OCEANIA' 'America ']


In [3]:
# Replace wrong values for the continent variable
data.loc[(data.continent == "América") |
         (data.continent == "AMERICA") |
         (data.continent == "AMERICa ") |
         (data.continent == "América ") |
         (data.continent == "America "), "continent"] = "America"

data.loc[(data.continent == "ASIA"), "continent"] = "Asia"

data.loc[(data.continent == "Europa "), "continent"] = "Europa"

data.loc[(data.continent == "Oceanía") |
         (data.continent == "Oceaniaa") |
         (data.continent == "OceANia") |
         (data.continent == "OCEANIA"), "continent"] = "Oceania"

data.loc[(data.continent == "africa") |
         (data.continent == "África") |
         (data.continent == "AFRICA"), "continent"] = "Africa"

# Check values
print(data["continent"].unique())

['Asia' 'Europa' 'Africa' 'America' 'Oceania']


**c)** Search unusual or anomalous values in the columns related to the alcohol consumption (beer, wine, spirit drinks and total).
- If the three alcoholic drinks are 0, the total must also be 0. 
- If the sum of the beer, wine and spirit drinks consumption is different than the total consumption of pure alcohol, the total's value must be replaced by `NaN`. 


In [4]:
# Display max and min of every variable
data.aggregate({"beer_servings": ["max", "min"],
                "spirit_servings": ["max", "min"],
                "wine_servings": ["max", "min"],
                "total_litres_of_pure_alcohol": ["max", "min"]})

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
max,376,438,370,100000.0
min,0,-50,-271,-8.2


In [5]:
# Replace negative values by NaN
data.spirit_servings = data.spirit_servings.apply(
    lambda x: np.nan if np.sign(x) < 0 else x)

data.wine_servings = data.wine_servings.apply(
    lambda x: np.nan if np.sign(x) < 0 else x)

data.total_litres_of_pure_alcohol = data.total_litres_of_pure_alcohol.apply(
    lambda x: np.nan if np.sign(x) < 0 else x)

In [6]:
data.aggregate({"beer_servings": ["max", "min"],
                "spirit_servings": ["max", "min"],
                "wine_servings": ["max", "min"],
                "total_litres_of_pure_alcohol": ["max", "min"]})

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
max,376,438.0,370.0,100000.0
min,0,0.0,0.0,0.0


In [7]:
# Determine the row of the max value in total variable
print(data.agg({"total_litres_of_pure_alcohol":["idxmax"]}))

        total_litres_of_pure_alcohol
idxmax                           147


In [8]:
# Replace the max with a NaN
data.loc[147, "total_litres_of_pure_alcohol"] = np.nan

data.aggregate({"beer_servings": ["max", "min"], 
                "spirit_servings": ["max", "min"], 
                "wine_servings": ["max", "min"], 
                "total_litres_of_pure_alcohol": ["max", "min"]})

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
max,376,438.0,370.0,14.4
min,0,0.0,0.0,0.0


In [9]:
# Visualize countries that a wrong sum
print(data[(data.beer_servings == 0) & (data.spirit_servings == 0) &
           (data.wine_servings == 0) & (data.total_litres_of_pure_alcohol != 0)])

print(data[(data.beer_servings != 0) & (data.spirit_servings != 0) &
           (data.wine_servings != 0) & (data.total_litres_of_pure_alcohol == 0)])

        country iso3 continent  beer_servings  spirit_servings  wine_servings  \
145  San Marino  SMR    Europa              0              0.0            0.0   

     total_litres_of_pure_alcohol  
145                          10.4  
        country iso3 continent  beer_servings  spirit_servings  wine_servings  \
114  Mozambique  MOZ    Africa             47             18.0            5.0   
116     Namibia  NAM    Africa            376              3.0            1.0   
149      Serbia  SRB    Europa            283            131.0          127.0   

     total_litres_of_pure_alcohol  
114                           0.0  
116                           0.0  
149                           0.0  


In [10]:
# Replace wrong values where the sum is wrong
data.loc[(data.beer_servings == 0) & (data.spirit_servings == 0) &
         (data.wine_servings == 0) & (data.total_litres_of_pure_alcohol != 0),
         'total_litres_of_pure_alcohol'] = 0.0
data.loc[(data.beer_servings != 0) & (data.spirit_servings != 0) &
         (data.wine_servings != 0) & (data.total_litres_of_pure_alcohol == 0),
         'total_litres_of_pure_alcohol'] = np.nan
# Check
print(data.loc[[145, 114, 116, 149]])

        country iso3 continent  beer_servings  spirit_servings  wine_servings  \
145  San Marino  SMR    Europa              0              0.0            0.0   
114  Mozambique  MOZ    Africa             47             18.0            5.0   
116     Namibia  NAM    Africa            376              3.0            1.0   
149      Serbia  SRB    Europa            283            131.0          127.0   

     total_litres_of_pure_alcohol  
145                           0.0  
114                           NaN  
116                           NaN  
149                           NaN  


**d)** Show all the `NaN` values and replace them by the median of their variable. Each country must use the median of their continents. 

In [11]:
# Find NaN values
print(data[data.isna().any(axis=1)])

               country iso3 continent  beer_servings  spirit_servings  \
98          Luxembourg  LUX    Europa            236            133.0   
109         Micronesia  FSM   Oceania             62              NaN   
114         Mozambique  MOZ    Africa             47             18.0   
116            Namibia  NAM    Africa            376              3.0   
128             Panama  PAN   America            285            104.0   
141  St. Kitts & Nevis  KNA   America            194            205.0   
147       Saudi Arabia  SAU      Asia              0              5.0   
149             Serbia  SRB    Europa            283            131.0   
157       South Africa  ZAF    Africa            225             76.0   

     wine_servings  total_litres_of_pure_alcohol  
98             NaN                          11.4  
109           18.0                           2.3  
114            5.0                           NaN  
116            1.0                           NaN  
128           

In [12]:
print("It is not necessary to transform beer_servings, spirit_servings, "
      "wine_servings and total_litres_of_pure_alcohol variable because " 
      "they already are numerical")
data.dtypes

It is not necessary to transform beer_servings, spirit_servings, wine_servings and total_litres_of_pure_alcohol variable because they already are numerical


country                          object
iso3                             object
continent                        object
beer_servings                     int64
spirit_servings                 float64
wine_servings                   float64
total_litres_of_pure_alcohol    float64
dtype: object

In [13]:
# Replace NaN values

data['spirit_servings'] = data['spirit_servings'].fillna(
    data.groupby('continent')['spirit_servings'].transform('median'))

data['wine_servings'] = data['wine_servings'].fillna(
    data.groupby('continent')['wine_servings'].transform('median'))

data['total_litres_of_pure_alcohol'] = data['total_litres_of_pure_alcohol'].fillna(
    data.groupby('continent')['total_litres_of_pure_alcohol'].transform('median'))

**e)** Check if there is any missing value

In [14]:
np.any(data.isna())

False

## Exercice 2


In this exercice the same dataset will be used.

**a)** Compute the following statistics: mean, maximum, minimum, quantiles (0.25, 0.5, 0.75) and standard deviation of each continent variables.

In [15]:
df = data.groupby('continent')

print('Asia')
df.get_group('Asia').describe()

Asia


,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,40.000000,40.000000,40.000000,40.000000
mean,36.000000,62.450000,13.675000,2.322500
std,44.438376,78.534967,33.942929,2.674906
min,0.000000,0.000000,0.000000,0.000000
25%,4.250000,1.000000,0.000000,0.200000
50%,20.500000,18.500000,1.000000,1.300000
75%,60.500000,101.000000,9.000000,4.000000
max,192.000000,258.000000,149.000000,9.800000


In [16]:
print('Africa')
df.get_group('Africa').describe()

Africa


,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,46.000000,46.000000,46.000000,46.000000
mean,64.391304,17.521739,17.673913,2.660870
std,86.106259,29.915243,41.453082,2.469231
min,0.000000,0.000000,0.000000,0.000000
25%,9.750000,1.000000,1.000000,0.625000
50%,27.000000,3.000000,2.000000,1.800000
75%,86.500000,24.250000,12.250000,4.175000
max,376.000000,152.000000,233.000000,9.100000


In [17]:
print('America')
df.get_group('America').describe()

America


,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,31.000000,31.000000,31.000000,31.000000
mean,146.838710,152.903226,31.193548,5.941935
std,70.068584,94.586241,50.886422,2.189638
min,1.000000,25.000000,1.000000,2.200000
25%,93.000000,92.000000,4.000000,4.200000
50%,149.000000,124.000000,9.000000,6.200000
75%,193.500000,177.000000,32.000000,7.200000
max,285.000000,438.000000,221.000000,11.900000


In [18]:
print('Europa')
df.get_group('Europa').describe()

Europa


,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,40.000000,40.00000,40.000000,40.000000
mean,206.400000,136.22500,145.300000,9.047500
std,95.088299,84.67812,95.601792,3.235222
min,0.000000,0.00000,0.000000,0.000000
25%,139.750000,79.50000,70.250000,6.675000
50%,230.500000,120.00000,128.000000,10.200000
75%,279.500000,178.25000,196.250000,11.300000
max,361.000000,373.00000,370.000000,14.400000


In [19]:
print('Oceania')
df.get_group('Oceania').describe()

Oceania


,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,12.000000,12.000000,12.000000,12.000000
mean,98.666667,53.333333,44.833333,3.675000
std,102.183732,68.323207,72.877024,3.525782
min,0.000000,0.000000,0.000000,0.000000
25%,38.250000,16.250000,1.000000,1.150000
50%,59.000000,35.000000,13.000000,2.150000
75%,129.500000,65.250000,36.500000,6.150000
max,306.000000,254.000000,212.000000,10.400000


**b)** To better compare the different measures of a variable, the data will be transformed to reduce the impact of the outlier values. Use [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling) to transform the dataset using a non-linear transformation to map the data to a uniform distribution.

In [20]:
# Use QuantileTransformer
import warnings
from sklearn.preprocessing import QuantileTransformer
warnings.simplefilter(action='ignore', category=Warning)

# Copy the data to transform
data_typ = data.copy()
s = QuantileTransformer()

# Transform
s.fit(data.iloc[:, 3:7])
data_typ.iloc[:, 3:7] = s.fit_transform(data_typ.iloc[:, 3:7])

# Check
data_typ.describe()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,169.000000,169.000000,169.000000,169.000000
mean,0.496883,0.492410,0.486211,0.497887
std,0.296775,0.303449,0.311715,0.295770
min,0.000000,0.000000,0.000000,0.000000
25%,0.252976,0.250000,0.235119,0.252976
50%,0.494048,0.500000,0.505952,0.494048
75%,0.750000,0.750000,0.750000,0.752976
max,1.000000,1.000000,1.000000,1.000000


**c)** The [WHO](https://www.who.int/es) (World Health Organization) wants to determine if the alcoholic drink that contributes more to the total of alcohol is the same in all continents. 

To do so, calculate the correlation of the variable `total_litres_of_pure_alcohol` with the rest of variables using the Pearson correlation coefficient and the transformed dataset. 

Which alcohol beverage contributes more to the total of alcohol? 

In [21]:
df = data_typ.groupby('continent')

In [22]:
df_Europa_c = df.get_group('Europa').corr(method="pearson")
df_Europa_c['total_litres_of_pure_alcohol'].sort_values(ascending=False)

total_litres_of_pure_alcohol    1.000000
beer_servings                   0.888234
wine_servings                   0.793350
spirit_servings                 0.735579
Name: total_litres_of_pure_alcohol, dtype: float64

In [23]:
df_Asia_c = df.get_group('Asia').corr(method="pearson")
df_Asia_c['total_litres_of_pure_alcohol'].sort_values(ascending=False)

total_litres_of_pure_alcohol    1.000000
beer_servings                   0.909307
spirit_servings                 0.776372
wine_servings                   0.747968
Name: total_litres_of_pure_alcohol, dtype: float64

In [24]:
df_Africa_c = df.get_group('Africa').corr(method="pearson")
df_Africa_c['total_litres_of_pure_alcohol'].sort_values(ascending=False)

total_litres_of_pure_alcohol    1.000000
beer_servings                   0.647150
wine_servings                   0.370464
spirit_servings                 0.359922
Name: total_litres_of_pure_alcohol, dtype: float64

In [25]:
df_America_c = df.get_group('America').corr(method="pearson")
df_America_c['total_litres_of_pure_alcohol'].sort_values(ascending=False)

total_litres_of_pure_alcohol    1.000000
wine_servings                   0.621048
spirit_servings                 0.513785
beer_servings                   0.509248
Name: total_litres_of_pure_alcohol, dtype: float64

In [26]:
df_Oceania_c = df.get_group('Oceania').corr(method="pearson")
df_Oceania_c['total_litres_of_pure_alcohol'].sort_values(ascending=False)

total_litres_of_pure_alcohol    1.000000
wine_servings                   0.906324
beer_servings                   0.716748
spirit_servings                 0.713820
Name: total_litres_of_pure_alcohol, dtype: float64

The beverage that most contributes to the total depends on the continent. Beer is the beverage that most contributes in Asia, Africa, Europe and in Oceania and America it is wine.  

**d)** The `continent` variable is not balanced, it doesn't contain the same number of observations for every possible categorical value. The classification algorithms tends to favor the more common values, thus the predictions would be biased. 

Determine the class with less observation and create a randomly balanced dataset using the number previously obtained.

In [27]:
# Determinem el nombre d'observacions per classe
print("Asia: {}".format(df.get_group('Asia').shape[0]))
print("America: {}".format(df.get_group('America').shape[0]))
print("Oceania: {}".format(df.get_group('Oceania').shape[0]))
print("Africa: {}".format(df.get_group('Africa').shape[0]))
print("Europa: {}".format(df.get_group('Europa').shape[0]))

Asia: 40
America: 31
Oceania: 12
Africa: 46
Europa: 40


In [28]:
print("The continent with less countries is Oceania. To obtain a balanced "
      "dataset, 12 countries must be randomly selected for each continent.")

The continent with less countries is Oceania. To obtain a balanced dataset, 12 countries must be randomly selected for each continent.


In [29]:
df_Asia = df.get_group('Asia').sample(n=12)
df_Africa = df.get_group('Africa').sample(n=12)
df_Europa = df.get_group('Europa').sample(n=12)
df_America = df.get_group('America').sample(n=12)

# Generem un únic dataframe amb el resultat
data_new = pd.concat([df_Asia, df_Africa, df_Europa,
                      df_America, df.get_group('Oceania')], ignore_index=True)
data_new

,country,iso3,continent,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Indonesia,IDN,Asia,0.133929,0.154762,0.000000,0.086310
1,Maldives,MDV,Asia,0.000000,0.000000,0.000000,0.000000
2,Kyrgyzstan,KGZ,Asia,0.333333,0.627976,0.410714,0.383929
3,Mongolia,MNG,Asia,0.494048,0.869048,0.470238,0.544643
4,Lebanon,LBN,Asia,0.252976,0.476190,0.660714,0.327381
5,South Korea,KOR,Asia,0.648810,0.330357,0.505952,0.839286
6,Bahrain,BHR,Asia,0.363095,0.508929,0.431548,0.339286
7,Kazakhstan,KAZ,Asia,0.619048,0.934524,0.565476,0.717262
8,Afghanistan,AFG,Asia,0.000000,0.000000,0.000000,0.000000
9,Azerbaijan,AZE,Asia,0.270833,0.464286,0.395833,0.252976


---

### Optional Exercise

Outliers are values that differ significantly from other observations and usually, they can distort the results. For this reason, it is important to discover these values and remove them from the analysis. Use the `IsolationForest` function from [sklearn.ensemble](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html) to determine the outliers considering that 20% of the dataset are outliers. 

How many countries are outliers? Which countries must be removed?

In [30]:
from sklearn.ensemble import IsolationForest

# Find the outliers
iforest = IsolationForest(contamination=0.2)
out = iforest.fit_predict(data.iloc[:, 3:7])

# Select the outliers (-1 if the country is outlier)
rowout = out != 1
data_out = data.iloc[rowout, 0]

print('Number of outlier countries: %d' % data_out.shape[0])
print(data_out)

Number of outlier countries: 34
3                  Andorra
6                Argentina
15                 Belarus
40            Cook Islands
45          Czech Republic
47                 Denmark
49                Dominica
54       Equatorial Guinea
60                  France
61                   Gabon
63                 Georgia
64                 Germany
66                  Greece
67                 Grenada
71                  Guyana
72                   Haiti
74                 Hungary
80                 Ireland
82                   Italy
91                    Laos
97               Lithuania
112             Montenegro
116                Namibia
127                  Palau
133                 Poland
134               Portugal
136            South Korea
139     Russian Federation
142              St. Lucia
146    Sao Tome & Principe
153               Slovakia
154               Slovenia
157           South Africa
164            Switzerland
Name: country, dtype: object
